# Training

In [ ]:
!cd ../bs-gym/ && pip install -e . -q
!cd .
!cd ../pytorch-a2c-ppo-acktr-gail/ && pip install -e . -q

In [ ]:
!which python
!which pip
!pip list

In [ ]:
import numpy as np
import time
from datetime import datetime
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt

from a2c_ppo_acktr.algo import PPO
from a2c_ppo_acktr.storage import RolloutStorage
from bs_gym.gymbattlesnake import BattlesnakeEnv

In [ ]:

print("Creating environment.")

from performance import check_performance
from policy import SnakePolicyBase, create_policy

from utils import n_envs, n_steps, n_opponents, CPU_THREADS, device
from utils import PathHelper

u = PathHelper()

tmp_env = BattlesnakeEnv(n_threads=CPU_THREADS, n_envs=n_envs)

# rollout storage: game turns played and rewards
rollouts = RolloutStorage(n_steps,
                          n_envs,
                          tmp_env.observation_space.shape,
                          tmp_env.action_space,
                          n_steps)
tmp_env.close()

policy = None
best_old_policy = None
agent = None

def setup_agent(model_path=None):
    global policy, best_old_policy, agent

    # policies
    policy = create_policy(tmp_env.observation_space.shape, tmp_env.action_space, SnakePolicyBase)
    # load latest model if found
    if model_path is not None:
        policy.load_state_dict(torch.load(model_path))

    best_old_policy = create_policy(tmp_env.observation_space.shape, tmp_env.action_space, SnakePolicyBase)
    best_old_policy.load_state_dict(policy.state_dict())

    # TODO: second_best model

    # agent
    # TODO: configure elsewhere
    agent = PPO(policy,
                value_loss_coef=0.5,
                entropy_coef=0.01,
                max_grad_norm=0.5,
                clip_param=0.2,
                ppo_epoch=4,
                num_mini_batch=16,
                eps=1e-5,
                lr=5e-5,
                )

env = None
obs = None

def setup_env():
    global env, obs
    
    # sanity check
    assert policy is not None
    assert best_old_policy is not None
    assert agent is not None

    # environment
    env = BattlesnakeEnv(n_threads=CPU_THREADS, n_envs=n_envs, opponents=[policy for _ in range(3)], device=device)
    # TODO: second_best model

    obs = env.reset()
    rollouts.obs[0].copy_(torch.tensor(obs))

    # send network/storage to gpu
    policy.to(device)
    best_old_policy.to(device)
    rollouts.to(device)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def plot_graphs(rewards, value_losses, lengths):
    plt.clf()

    plt.title("Average Length vs Episode")
    plt.ylabel("Length")
    plt.xlabel("Iteration")
    plt.plot(lengths)
    plt.show()

    plt.title("Average Loss vs Episode")
    plt.ylabel("Length")
    plt.xlabel("Iteration")
    plt.plot(lengths)
    plt.show(value_losses)

    plt.title("Average Reward vs Episode")
    plt.ylabel("Reward")
    plt.xlabel("Iteration")
    plt.plot(rewards)
    plt.show()

In [ ]:
last_iteration = -1

rewards = []
value_losses = []
lengths = []

def train(num_updates, start_iteration=0, test_every=5):
    print("Starting training.")

    start = time.time()
    for i in range(num_updates):
        j = start_iteration + i
        episode_rewards = []
        episode_lengths = []
        
        policy.eval()
        print(f"Iteration {j}: Generating rollouts")
        for step in tqdm(range(n_steps)):
            with torch.no_grad():
                value, action, action_log_prob, recurrent_hidden_states = policy.act(rollouts.obs[step],
                                                                rollouts.recurrent_hidden_states[step],
                                                                rollouts.masks[step])
            obs, reward, done, infos = env.step(action.cpu().squeeze())
            obs = torch.tensor(obs)
            reward = torch.tensor(reward).unsqueeze(1)

            for info in infos:
                if 'episode' in info.keys():
                    episode_rewards.append(info['episode']['r'])
                    episode_lengths.append(info['episode']['l'])

            masks = torch.FloatTensor([[0.0] if done_ else [1.0] for done_ in done])
            bad_masks = torch.FloatTensor([[0.0] if 'bad_transition' in info.keys() else [1.0] for info in infos])
            rollouts.insert(obs, recurrent_hidden_states, action, action_log_prob, value, reward, masks, bad_masks)

        with torch.no_grad():
            next_value = policy.get_value(
                rollouts.obs[-1],
                rollouts.recurrent_hidden_states[-1],
                rollouts.masks[-1]
            ).detach()
            
        policy.train()

        print("Training policy on rollouts...")
        # We're using a gamma = 0.99 and lambda = 0.95
        rollouts.compute_returns(next_value, True, 0.99, 0.95, False)
        value_loss, action_loss, dist_entropy = agent.update(rollouts)
        rollouts.after_update()

        policy.eval()
        
        total_num_steps = (j + 1) * n_envs * n_steps
        end = time.time()
        
        lengths.append(np.mean(episode_lengths))
        rewards.append(np.mean(episode_rewards))
        value_losses.append(value_loss)
        
        # TODO: modify 5 to be a variable
        if j % check_every == 0 and j != 0:
            print("\n")
            print("=" * 80)
            print("Iteration", j, "Results")
            # TODO: do in parallel
            # Check the performance of the current policy against the prior best
            winrate = check_performance(policy, best_old_policy, device=torch.device(device))#device=device)
            print(f"Winrate vs prior best: {winrate*100:.2f}%")
            print(f"Median Length: {np.median(episode_lengths)}")
            print(f"Max Length: {np.max(episode_lengths)}")
            print(f"Min Length: {np.min(episode_lengths)}")

            # TODO: modify winrate requirement to be a variable, eg. 30% (for 4 players)
            if winrate > 0.3:
                print("Policy winrate is > 30%. Updating prior best model")
                best_old_policy.load_state_dict(policy.state_dict())
                print("Saving latest best model.")
                
                torch.save(best_old_policy.state_dict(), u.get_modelpath(iteration=j))
            else:
                print("Policy has not learned enough yet... keep training!")
            print("-" * 80)
            
    print("Saving final model.")
    last_iteration = start_iteration + num_updates - 1
    torch.save(policy.state_dict(), u.get_modelpath_latest())

    return rewards, value_losses, lengths

In [ ]:
# Do matmul at TF32 mode.
torch.backends.cuda.matmul.allow_tf32 = False

u.set_modelgroup('test')

num_updates = 5 # iterations to run
# num_updates = 4200 # for competition training

latest_model_path, iteration = u.get_latest_model()
print(f"Latest model: {latest_model_path}")
print(f"Latest iteration: {iteration}")

setup_agent(model_path=latest_model_path)
setup_env()


rewards2, value_losses2, lengths2 = train(num_updates, start_iteration=iteration+1
                                          test_every=10
                                          )
# TODO: send notification when complete

In [ ]:
print(f"Trainable Parameters: {count_parameters(policy)}")
# TODO: last model
# TODO: iterations


plot_graphs(rewards, value_losses, lengths)